In [20]:
import sys
import os
import argparse
import json
import re
import tokenize
import io
from io import BytesIO

In [9]:
text = "Hello, how are you doing?"
tokens = text.split()  # Split text by spaces
print(tokens)

text = "Hello, how are you doing?"
tokens = re.findall(r'\b\w+\b', text)  # Find all words
print(tokens)

['Hello,', 'how', 'are', 'you', 'doing?']
['Hello', 'how', 'are', 'you', 'doing']


In [10]:
# Python code as a string
code = "import sys\nprint(sys.version)"

# Convert the string code to bytes and wrap in a BytesIO stream
bytes_io = io.BytesIO(code.encode('utf-8'))

# Use tokenize to read the tokens
for token in tokenize.tokenize(bytes_io.readline):
    print(token)

TokenInfo(type=63 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line='')
TokenInfo(type=1 (NAME), string='import', start=(1, 0), end=(1, 6), line='import sys\n')
TokenInfo(type=1 (NAME), string='sys', start=(1, 7), end=(1, 10), line='import sys\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(1, 10), end=(1, 11), line='import sys\n')
TokenInfo(type=1 (NAME), string='print', start=(2, 0), end=(2, 5), line='print(sys.version)')
TokenInfo(type=54 (OP), string='(', start=(2, 5), end=(2, 6), line='print(sys.version)')
TokenInfo(type=1 (NAME), string='sys', start=(2, 6), end=(2, 9), line='print(sys.version)')
TokenInfo(type=54 (OP), string='.', start=(2, 9), end=(2, 10), line='print(sys.version)')
TokenInfo(type=1 (NAME), string='version', start=(2, 10), end=(2, 17), line='print(sys.version)')
TokenInfo(type=54 (OP), string=')', start=(2, 17), end=(2, 18), line='print(sys.version)')
TokenInfo(type=4 (NEWLINE), string='', start=(2, 18), end=(2, 19), line='')
TokenInfo(type=0 (ENDMA

In [32]:
import ast

def convert_file(in_file_name, output_dir):
    base_file_name = os.path.splitext(os.path.basename(in_file_name))[0]
    out_file_name = os.path.join(output_dir, base_file_name + ".json")
    with open(in_file_name) as in_fs:
        lines = in_fs.readlines()
        
    text_str = "".join(lines)
    tokens = list(tokenize.tokenize(BytesIO(text_str.encode('utf-8')).readline))
    print(tokens)
    docstring = None
    if tokens[1].type == 3:
        docstring = tokens[1].string
    
    out_dict = {"docstring" : docstring,
                "map_and_model" : None,
                "constants" : None,
                "monitors" : None,
                "behaviors" : None,
                "spatial_relations" : None,
                "scenario" : None,
                "background" : None}

    current_doc = None
    line = 1
    for token in tokens:
        if token.type == 61: # Comment Type
            if "map" in token.string.lower():
                current_doc = "map_and_model"
            elif "constants" in token.string.lower():
                current_doc = "constants"
            elif "monitors" in token.string.lower():
                current_doc = "monitors"
            elif "behaviors" in token.string.lower():
                current_doc = "behaviors"
            elif "spatial relations" in token.string.lower():
                current_doc = "spatial_relations"
            elif "scenario" in token.string.lower():
                current_doc = "scenario"
            elif "background" in token.string.lower():
                current_doc = "background"
        else:
            if current_doc is not None and out_dict[current_doc] is not None and token.start[0] != line:
                out_dict[current_doc] += token.line
            elif current_doc is not None and out_dict[current_doc] is None and token.start[0] != line:
                out_dict[current_doc] = token.string
        line = token.end[0]

    with open(out_file_name, "w") as out_fs:
        json.dump(out_dict, out_fs, indent=2)

convert_file("/home/xiaoyc/Jaredan/SceGenWithLLM/data_scenic/Carla_Challenge/carlaChallenge1.scenic", "./")



[TokenInfo(type=63 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line=''), TokenInfo(type=3 (STRING), string='""" Scenario Description\nTraffic Scenario 01.\nControl loss without previous action.\nThe ego-vehicle loses control due to bad conditions on the road and it must recover, coming back to\nits original lane.\n"""', start=(1, 0), end=(6, 3), line='""" Scenario Description\nTraffic Scenario 01.\nControl loss without previous action.\nThe ego-vehicle loses control due to bad conditions on the road and it must recover, coming back to\nits original lane.\n"""\n'), TokenInfo(type=4 (NEWLINE), string='\n', start=(6, 3), end=(6, 4), line='"""\n'), TokenInfo(type=62 (NL), string='\n', start=(7, 0), end=(7, 1), line='\n'), TokenInfo(type=61 (COMMENT), string='## SET MAP AND MODEL (i.e. definitions of all referenceable vehicle types, road library, etc)', start=(8, 0), end=(8, 93), line='## SET MAP AND MODEL (i.e. definitions of all referenceable vehicle types, road library, etc)\n'